In [1]:
import torch, os, copy, time
from tqdm import tqdm
from torch import nn
from torch.utils.data import DataLoader, Dataset
from load_adult import *
import torch.nn.functional as F
from torch.autograd import Variable
from functools import partial
import pandas as pd

In [3]:
class logReg(torch.nn.Module):
    # logistic regression 
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.linear = torch.nn.Linear(num_features, num_classes)

    def forward(self, x):
        logits = self.linear(x.float())
        probas = torch.sigmoid(logits)
        return probas.type(torch.FloatTensor), logits

In [3]:
def loss_func(option, logits, targets, distance, sensitive, mean_sensitive, larg = 1):
    acc_loss = F.cross_entropy(logits, targets, reduction = 'sum')
    fair_loss = torch.mul(sensitive - sensitive.type(torch.FloatTensor).mean(), distance.T[0])
    fair_loss = torch.mean(torch.mul(fair_loss, fair_loss)) # modified mean to sum
    if option == 'unconstrained':
        return acc_loss, acc_loss, larg*fair_loss
    if option == 'Zafar':
        return acc_loss + larg*fair_loss, acc_loss, larg*fair_loss

In [4]:
class DatasetSplit(Dataset):
    """An abstract Dataset class wrapped around Pytorch Dataset class.
    """

    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        feature, label, sensitive = self.dataset[self.idxs[item]]
        return feature, label, sensitive


class ClientUpdate(object):
    def __init__(self, dataset, idxs, batch_size, option, penalty = 0):
        self.trainloader, self.validloader = self.train_val(dataset, list(idxs), batch_size)
        self.option = option
        self.penalty = penalty
            
    def train_val(self, dataset, idxs, batch_size):
        """
        Returns train, validation for a given local training dataset
        and user indexes.
        """
        # split indexes for train, validation (90, 10)

        idxs_train = idxs[:int(0.9*len(idxs))]
        idxs_val = idxs[int(0.9*len(idxs)):len(idxs)]

        trainloader = DataLoader(DatasetSplit(dataset, idxs_train),
                                 batch_size=batch_size, shuffle=True)
        validloader = DataLoader(DatasetSplit(dataset, idxs_val),
                                 batch_size=int(len(idxs_val)/10), shuffle=False)
        return trainloader, validloader

    def update_weights(self, model, global_round, learning_rate, local_epochs, optimizer):
        # Set mode to train model
        model.train()
        epoch_loss = []

        # Set optimizer for the local updates
        if optimizer == 'sgd':
            optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,
                                        ) # momentum=0.5
        elif optimizer == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                                         weight_decay=1e-4)

        for i in range(local_epochs):
            batch_loss = []
            for batch_idx, (features, labels, sensitive) in enumerate(self.trainloader):
                features, labels = features.to(DEVICE), labels.to(DEVICE).type(torch.LongTensor)
                # we need to set the gradients to zero before starting to do backpropragation 
                # because PyTorch accumulates the gradients on subsequent backward passes. 
                # This is convenient while training RNNs
                
                log_probs, logits = model(features)
                loss, _, _ = loss_func(self.option,
                    logits, labels, logits, sensitive, mean_sensitive, self.penalty)
                    
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if batch_idx % 50 == 0:
                    print('| Global Round : {} | Local Epoch : {} | [{}/{} ({:.0f}%)]\tBatch Loss: {:.6f}'.format(
                        global_round, i, batch_idx * len(features),
                        len(self.trainloader.dataset),
                        100. * batch_idx / len(self.trainloader), loss.item()))
                batch_loss.append(loss.item())
            epoch_loss.append(sum(batch_loss)/len(batch_loss))

        # weight, loss
        return model.state_dict(), sum(epoch_loss) / len(epoch_loss)

    def inference(self, model):
        """ 
        Returns the inference accuracy, 
                                loss, 
                                N(sensitive group, pos), 
                                N(non-sensitive group, pos), 
                                N(sensitive group),
                                N(non-sensitive group),
                                acc_loss,
                                fair_loss
        """

        model.eval()
        loss, total, correct, fair_loss, acc_loss, num_batch = 0.0, 0.0, 0.0, 0.0, 0.0, 0
        sp, nsp, s, n = 0, 0, 0, 0
        for batch_idx, (features, labels, sensitive) in enumerate(self.validloader):
            features, labels = features.to(DEVICE), labels.to(DEVICE).type(torch.LongTensor)
            sensitive = sensitive.to(DEVICE)
            
            # Inference
            outputs, logits = model(features)

            # Prediction
            _, pred_labels = torch.max(outputs, 1)
            pred_labels = pred_labels.view(-1)
            bool_correct = torch.eq(pred_labels, labels)
            correct += torch.sum(bool_correct).item()
            total += len(labels)
            num_batch += 1
            bool_sensitive = torch.eq(sensitive, torch.ones(len(labels)))
            s += torch.sum(bool_sensitive).item()
            n += torch.sum(torch.logical_not(bool_sensitive)).item()
            sp += torch.sum(torch.logical_and(pred_labels, bool_sensitive)).item()
            nsp += torch.sum(torch.logical_and(pred_labels, torch.logical_not(bool_sensitive))).item()
            
            batch_loss, batch_acc_loss, batch_fair_loss = loss_func(self.option, outputs, 
                                                        labels, logits, sensitive, mean_sensitive, self.penalty)
            loss, acc_loss, fair_loss = (loss + batch_loss.item(), 
                                         acc_loss + batch_acc_loss.item(), 
                                         fair_loss + batch_fair_loss.item())
        accuracy = correct/total
        return accuracy, loss, s, n, sp, nsp, acc_loss / num_batch, fair_loss / num_batch


def test_inference(model, test_dataset, batch_size):
    """ Returns the test accuracy and loss.
    """

    model.eval()
    loss, total, correct = 0.0, 0.0, 0.0
    sp, nsp, s, n = 0, 0, 0, 0
    
    criterion = nn.NLLLoss().to(DEVICE)
    testloader = DataLoader(test_dataset, batch_size=batch_size,
                            shuffle=False)

    for batch_idx, (features, labels, sensitive) in enumerate(testloader):
        features = features.to(DEVICE)
        labels =  labels.to(DEVICE).type(torch.LongTensor)
        # Inference
        outputs, logits = model(features)
        batch_loss = criterion(outputs, labels)
        loss += batch_loss.item()

        # Prediction
        _, pred_labels = torch.max(outputs, 1)
        pred_labels = pred_labels.view(-1)
        bool_correct = torch.eq(pred_labels, labels)
        correct += torch.sum(bool_correct).item()
        total += len(labels)

        bool_sensitive = torch.eq(sensitive, torch.ones(len(labels)))
        s += torch.sum(bool_sensitive).item()
        n += torch.sum(torch.logical_not(bool_sensitive) ).item()
        sp += torch.sum(torch.logical_and(bool_correct, bool_sensitive)).item()
        nsp += torch.sum(torch.logical_and(bool_correct, torch.logical_not(bool_sensitive))).item()

    accuracy = correct/total
    # |P(Group1, pos) - P(Group2, pos)| = |N(Group1, pos)/N(Group1) - N(Group2, pos)/N(Group2)|
    return accuracy, loss, abs(sp/s-nsp/n)

In [5]:
def train(model, option = "unconstrained", batch_size = 128, num_clients = 2
          num_rounds = 5, learning_rate = 0.01, optimizer = 'adam', local_epochs= 5, 
          num_workers = 4, print_every = 1,
         penalty = 1):
    # Training
    train_loss, train_accuracy = [], []
    val_acc_list, net_list = [], []
    cv_loss, cv_acc = [], []
    val_loss_pre, counter = 0, 0
    start_time = time.time()
    weights = model.state_dict()
    
    test_loader = DataLoader(dataset = test_dataset,
                            batch_size = batch_size,
                            num_workers = num_workers)
    
    train_loader = DataLoader(dataset = train_dataset,
                        batch_size = batch_size,
                        num_workers = num_workers)

    def average_weights(w):
        """
        Returns the average of the weights.
        """
        w_avg = copy.deepcopy(w[0])
        for key in w_avg.keys():
            for i in range(1, len(w)):
                w_avg[key] += w[i][key]
            w_avg[key] = torch.div(w_avg[key], len(w))
        return w_avg

    # lambda
    if option == 'FairBatch':
        lbd = [((adult.salary == 0) & (adult[sen_var] == 0)).mean(), 
               ((adult.salary == 0) & (adult[sen_var] == 1)).mean()]
    
    for round_ in tqdm(range(num_rounds)):
        local_weights, local_losses = [], []
        print(f'\n | Global Training Round : {round_+1} |\n')

        model.train()
        m = 2 # the number of clients to be chosen in each round_
        idxs_users = np.random.choice(range(num_clients), m, replace=False)

        for idx in idxs_users:
            local_model = ClientUpdate(dataset=train_dataset,
                                        idxs=clients_idx[idx], batch_size = batch_size, 
                                       option = option, penalty = penalty)
            w, loss = local_model.update_weights(
                            model=copy.deepcopy(model), global_round=round_, 
                                learning_rate = learning_rate, local_epochs = local_epochs, 
                                optimizer = optimizer, lbd = lbd)
            local_weights.append(copy.deepcopy(w))
            local_losses.append(copy.deepcopy(loss))

        # update global weights
        weights = average_weights(local_weights)
        model.load_state_dict(weights)
        

        loss_avg = sum(local_losses) / len(local_losses)
        train_loss.append(loss_avg)

        # Calculate avg training accuracy over all users at every round
        list_acc, list_loss = [], []
        s, n, sp, nsp = 0, 0, 0, 0
        model.eval()
        for c in range(m):
            local_model = ClientUpdate(dataset=train_dataset,
                                        idxs=clients_idx[c], batch_size = batch_size, option = option, penalty = penalty)
            # validation dataset inference
            acc, loss, s_, n_, sp_, nsp_, acc_loss, fair_loss = local_model.inference(model=model) 
            list_acc.append(acc)
            list_loss.append(loss)
            s, n, sp, nsp = s + s_, n + n_, sp + sp_, nsp + nsp_
            print("Client %d: accuracy loss: %.2f | fairness loss %.2f | RD = %.2f = |%d/%d-%d/%d| " % (
                c, acc_loss, fair_loss, abs(sp_/s_-nsp_/n_), sp_, s_, nsp_, n_))
            
        train_accuracy.append(sum(list_acc)/len(list_acc))

        # print global training loss after every 'i' rounds
        if (round_+1) % print_every == 0:
            print(f' \nAvg Training Stats after {round_+1} global rounds:')
            print("Training loss: %.2f | Validation accuracy: %.2f%% | Validation RD: %.2f" % (
                 np.mean(np.array(train_loss)), 
                100*train_accuracy[-1],
                abs(sp/s-nsp/n)
                 ))

    # Test inference after completion of training
    test_acc, test_loss, rd= test_inference(model, test_dataset, batch_size)

    print(f' \n Results after {num_rounds} global rounds of training:')
    print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
    print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

    # Compute RD: risk difference - fairness metric
    # |P(Group1, pos) - P(Group2, pos)| = |N(Group1, pos)/N(Group1) - N(Group2, pos)/N(Group2)|
    print("|---- Test RD: {:.2f}".format(rd))

    print('\n Total Run Time: {0:0.4f} sec'.format(time.time()-start_time))

In [6]:
train(logReg(num_features=NUM_FEATURES, num_classes=2), 
      "Zafar", penalty = 50, optimizer = 'sgd', learning_rate = 0.01,
     num_rounds = 5, local_epochs = 10)

  0%|          | 0/5 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 97.129074
| Global Round : 0 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 88.739372
| Global Round : 0 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 58.027599
| Global Round : 0 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 73.519951
| Global Round : 0 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 63.784790
| Global Round : 0 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 75.687607
| Global Round : 0 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 51.680065
| Global Round : 0 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 74.850227
| Global Round : 0 | Local Epoch : 4 | [0/11188 (0%)]	Batch Loss: 60.017704
| Global Round : 0 | Local Epoch : 4 | [6400/11188 (57%)]	Batch Loss: 75.023788
| Global Round : 0 | Local Epoch : 5 | [0/11188 (0%)]	Batch Loss: 40.387955
| Global Round : 0 | Local Epoch : 5 | [6400/11188 (57%)]	Batch Loss: 50.170868
| Global Round : 0 | Local Epoc

 20%|██        | 1/5 [00:07<00:31,  7.96s/it]

Client 0: accuracy loss: 115.47 | fairness loss 5.94 | RD = 0.38 = |140/592-877/1421| 
Client 1: accuracy loss: 69.04 | fairness loss 7.19 | RD = 0.30 = |88/421-423/823| 
 
Avg Training Stats after 1 global rounds:
Training loss: 81.70 | Validation accuracy: 75.11% | Validation RD: 0.35

 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 74.821060
| Global Round : 1 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 68.413315
| Global Round : 1 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 52.999275
| Global Round : 1 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 38.131889
| Global Round : 1 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 56.216457
| Global Round : 1 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 50.668480
| Global Round : 1 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 45.660702
| Global Round : 1 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 52.844872
| Global Round : 1 | Local Epoch : 4 | [0/11188 (0%)]	B

 40%|████      | 2/5 [00:16<00:24,  8.16s/it]

...

 80%|████████  | 4/5 [00:33<00:08,  8.23s/it]

Client 0: accuracy loss: 105.01 | fairness loss 0.25 | RD = 0.26 = |80/592-563/1421| 
Client 1: accuracy loss: 63.38 | fairness loss 0.24 | RD = 0.25 = |46/421-296/823| 
 
Avg Training Stats after 4 global rounds:
Training loss: 72.13 | Validation accuracy: 83.13% | Validation RD: 0.26

 | Global Training Round : 5 |

| Global Round : 4 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 41.820972
| Global Round : 4 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 58.578114
| Global Round : 4 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 52.752613
| Global Round : 4 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 65.227402
| Global Round : 4 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 41.261173
| Global Round : 4 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 54.598175
| Global Round : 4 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 49.818287
| Global Round : 4 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 87.625832
| Global Round : 4 | Local Epoch : 4 | [0/11188 (0%)]	Ba

100%|██████████| 5/5 [00:40<00:00,  8.18s/it]

Client 1: accuracy loss: 62.64 | fairness loss 1.26 | RD = 0.23 = |42/421-274/823| 
 
Avg Training Stats after 5 global rounds:
Training loss: 71.22 | Validation accuracy: 83.88% | Validation RD: 0.25


 
 Results after 5 global rounds of training:
|---- Avg Train Accuracy: 83.88%
|---- Test Accuracy: 84.48%
|---- Test RD: 0.12

 Total Run Time: 41.2722 sec


In [7]:
train(logReg(num_features=NUM_FEATURES, num_classes=2), optimizer = 'sgd', learning_rate = 0.01)

  0%|          | 0/5 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 93.002548
| Global Round : 0 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 69.492393
| Global Round : 0 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 47.031193
| Global Round : 0 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 54.340210
| Global Round : 0 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 57.740227
| Global Round : 0 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 42.863007
| Global Round : 0 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 35.554657
| Global Round : 0 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 37.808662
| Global Round : 0 | Local Epoch : 4 | [0/11188 (0%)]	Batch Loss: 40.819706
| Global Round : 0 | Local Epoch : 4 | [6400/11188 (57%)]	Batch Loss: 60.515236
| Global Round : 0 | Local Epoch : 0 | [0/18116 (0%)]	Batch Loss: 93.193375
| Global Round : 0 | Local Epoch : 0 | [6400/18116 (35%)]	Batch Loss: 61.420052
| Global Round : 0 | Local Epoc

 20%|██        | 1/5 [00:03<00:15,  3.80s/it]

Client 0: accuracy loss: 105.00 | fairness loss 0.37 | RD = 0.38 = |104/592-784/1421| 
Client 1: accuracy loss: 61.40 | fairness loss 0.52 | RD = 0.29 = |62/421-363/823| 
 
Avg Training Stats after 1 global rounds:
Training loss: 48.38 | Validation accuracy: 78.70% | Validation RD: 0.35

 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/18116 (0%)]	Batch Loss: 27.846607
| Global Round : 1 | Local Epoch : 0 | [6400/18116 (35%)]	Batch Loss: 48.043636
| Global Round : 1 | Local Epoch : 0 | [12800/18116 (70%)]	Batch Loss: 61.422527
| Global Round : 1 | Local Epoch : 1 | [0/18116 (0%)]	Batch Loss: 40.716099
| Global Round : 1 | Local Epoch : 1 | [6400/18116 (35%)]	Batch Loss: 47.060272
| Global Round : 1 | Local Epoch : 1 | [12800/18116 (70%)]	Batch Loss: 39.354794
| Global Round : 1 | Local Epoch : 2 | [0/18116 (0%)]	Batch Loss: 49.975140
| Global Round : 1 | Local Epoch : 2 | [6400/18116 (35%)]	Batch Loss: 38.019928
| Global Round : 1 | Local Epoch : 2 | [12800/181

 40%|████      | 2/5 [00:08<00:11,  3.94s/it]

...

 80%|████████  | 4/5 [00:16<00:04,  4.08s/it]

Client 1: accuracy loss: 59.30 | fairness loss 0.61 | RD = 0.20 = |45/421-250/823| 
 
Avg Training Stats after 4 global rounds:
Training loss: 45.98 | Validation accuracy: 83.74% | Validation RD: 0.20

 | Global Training Round : 5 |

| Global Round : 4 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 37.388119
| Global Round : 4 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 57.172165
| Global Round : 4 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 60.511341
| Global Round : 4 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 54.603451
| Global Round : 4 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 48.786533
| Global Round : 4 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 40.720695
| Global Round : 4 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 49.488594
| Global Round : 4 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 50.179955
| Global Round : 4 | Local Epoch : 4 | [0/11188 (0%)]	Batch Loss: 50.713879
| Global Round : 4 | Local Epoch : 4 | [6400/11188 (57%)]	Batch Lo

100%|██████████| 5/5 [00:20<00:00,  4.13s/it]

Client 0: accuracy loss: 98.95 | fairness loss 0.55 | RD = 0.24 = |76/592-521/1421| 
Client 1: accuracy loss: 59.18 | fairness loss 0.64 | RD = 0.24 = |44/421-283/823| 
 
Avg Training Stats after 5 global rounds:
Training loss: 45.62 | Validation accuracy: 83.51% | Validation RD: 0.24


 
 Results after 5 global rounds of training:
|---- Avg Train Accuracy: 83.51%
|---- Test Accuracy: 84.52%
|---- Test RD: 0.12

 Total Run Time: 21.0496 sec


In [5]:
((adult.salary == 0) & (adult[sen_var] == 0)).mean()

0.036208961641227236

In [6]:
((adult.salary == 0) & (adult[sen_var] == 1)).mean()

0.29458554712693097